### Analisis exploratorio de datos

In [2]:
import pandas as pd 
import numpy as np 
import seaborn as sb


In [3]:
data = pd.read_csv("df_final.csv")

C:\Users\Jose Quispe}\AppData\Local\Temp\ipykernel_22744\1599027352.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("df_final.csv")


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45379 entries, 0 to 45378
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   budget                 45379 non-null  object 
 1   id                     45379 non-null  object 
 2   original_language      45368 non-null  object 
 3   overview               44438 non-null  object 
 4   popularity             45377 non-null  object 
 5   release_date           45376 non-null  object 
 6   revenue                45379 non-null  float64
 7   runtime                45130 non-null  float64
 8   status                 45296 non-null  object 
 9   tagline                20398 non-null  object 
 10  title                  45376 non-null  object 
 11  vote_average           45376 non-null  float64
 12  vote_count             45376 non-null  float64
 13  belongs_to_collection  45379 non-null  object 
 14  genres                 45379 non-null  object 
 15  pr

In [4]:
# Eliminamos las columnas que no nos sirven
data = data.drop(['budget', 'release_date', 'status', 'vote_count', 'belongs_to_collection',
                  'spoken_languages', 'tagline'], axis=1)

In [5]:
data = data.dropna(subset=['overview'])

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44438 entries, 0 to 45378
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    44438 non-null  object 
 1   original_language     44428 non-null  object 
 2   overview              44438 non-null  object 
 3   popularity            44436 non-null  object 
 4   revenue               44438 non-null  float64
 5   runtime               44435 non-null  float64
 6   title                 44435 non-null  object 
 7   vote_average          44435 non-null  float64
 8   genres                44438 non-null  object 
 9   production_companies  44438 non-null  object 
 10  production_countries  44438 non-null  object 
 11  release_year          44438 non-null  float64
 12  return                44438 non-null  float64
dtypes: float64(5), object(8)
memory usage: 4.7+ MB


Usaremos las siguientes columnas como filtros para eliminar peliculas no interesantes:
* popularity 
* revenue
* vote_average
* release_year
* original_language
* runtime
* production_countries

In [6]:
data['popularity'] = pd.to_numeric(data['popularity'], errors='coerce')

In [7]:
# Eliminamos los valores por debajo de la mediana ya que el histograma tiene sesgo positivo
threshold = data['popularity'].median() 
data = data[data['popularity'] >= threshold]

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22218 entries, 0 to 45374
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    22218 non-null  object 
 1   original_language     22217 non-null  object 
 2   overview              22218 non-null  object 
 3   popularity            22218 non-null  float64
 4   revenue               22218 non-null  float64
 5   runtime               22218 non-null  float64
 6   title                 22218 non-null  object 
 7   vote_average          22218 non-null  float64
 8   genres                22218 non-null  object 
 9   production_companies  22218 non-null  object 
 10  production_countries  22218 non-null  object 
 11  release_year          22218 non-null  float64
 12  return                22218 non-null  float64
dtypes: float64(6), object(7)
memory usage: 2.4+ MB


In [174]:
print(data['original_language'].unique())

['en' 'fr' 'it' 'fa' 'nl' 'cn' 'ru' 'zh' 'es' 'ja' 'sv' 'sr' 'bn' 'de'
 'vi' 'cs' 'da' 'pt' 'no' 'pl' 'el' 'sh' 'xx' 'ko' 'ca' 'fi' 'sk' 'th'
 'hu' 'bs' 'hi' 'tr' 'is' 'he' 'ps' 'ab' 'mn' 'zu' 'uk' 'af' 'bo' 'ro'
 'nb' 'ar' 'et' 'tl' 'bm' 'bg' 'id' 'ta' 'te' 'sq' 'ku' 'lo' 'ka' 'mk'
 'ml' 'lt' 'ne' 'lv' 'sl' 'ky' 'ms' 'ur' 'hr' 'mr' nan]


In [10]:
language_counts = data['original_language'].value_counts()
language_counts.head(10)

original_language
en    16725
fr     1296
ja      692
it      534
de      405
es      371
ko      276
ru      269
hi      242
zh      217
Name: count, dtype: int64

In [9]:
# Definimos la lista de idiomas que queremos conservar
languages_to_keep = ['en', 'fr', 'ja', 'it', 'es', 'ko', 'ru']

# Filtramos el dataframe para tener solo las peliculas con los idiomas mas hablados
data = data[data['original_language'].isin(languages_to_keep)]


In [176]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20568 entries, 0 to 45374
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    20568 non-null  object 
 1   original_language     20568 non-null  object 
 2   overview              20568 non-null  object 
 3   popularity            20568 non-null  float64
 4   revenue               20568 non-null  float64
 5   runtime               20568 non-null  float64
 6   title                 20568 non-null  object 
 7   vote_average          20568 non-null  float64
 8   genres                20568 non-null  object 
 9   production_companies  20568 non-null  object 
 10  production_countries  20568 non-null  object 
 11  release_year          20568 non-null  float64
 12  return                20568 non-null  float64
dtypes: float64(6), object(7)
memory usage: 2.2+ MB


In [13]:
# Obtener el conteo de valores float
conteo = data['release_year'].value_counts()

conteo.head(60)

release_year
2014.0    1016
2013.0     988
2015.0     964
2012.0     848
2016.0     843
2011.0     802
2009.0     779
2008.0     748
2010.0     706
2006.0     641
2007.0     629
2005.0     539
2004.0     480
2002.0     427
2003.0     417
2001.0     412
2000.0     359
2017.0     353
1999.0     349
1998.0     340
1996.0     338
1997.0     309
1995.0     283
1994.0     277
1993.0     258
1988.0     245
1991.0     232
1992.0     227
1990.0     224
1987.0     223
1989.0     214
1985.0     198
1986.0     196
1984.0     172
1983.0     164
1982.0     161
1981.0     159
1980.0     157
1973.0     143
1971.0     138
1972.0     137
1978.0     129
1979.0     128
1974.0     127
1966.0     124
1975.0     120
1976.0     120
1977.0     119
1970.0     117
1967.0     115
1968.0     113
1964.0     109
1969.0     102
1965.0      97
1959.0      92
1962.0      91
1960.0      91
1963.0      88
1957.0      79
1955.0      79
Name: count, dtype: int64

In [10]:
data = data[data['release_year'] >= 1960]

In [161]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18961 entries, 0 to 45374
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    18961 non-null  object 
 1   original_language     18961 non-null  object 
 2   overview              18961 non-null  object 
 3   popularity            18961 non-null  float64
 4   revenue               18961 non-null  float64
 5   runtime               18961 non-null  float64
 6   title                 18961 non-null  object 
 7   vote_average          18961 non-null  float64
 8   genres                18961 non-null  object 
 9   production_companies  18961 non-null  object 
 10  production_countries  18961 non-null  object 
 11  release_year          18961 non-null  float64
 12  return                18961 non-null  float64
dtypes: float64(6), object(7)
memory usage: 2.0+ MB


In [11]:
data = data[data['vote_average'] >= 5]

In [12]:
data = data[data['overview'].apply(lambda x: len(str(x).split()) >= 50)]
data = data[data['overview'].apply(lambda x: len(str(x).split()) <= 200)]

In [202]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8277 entries, 0 to 45374
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    8277 non-null   object 
 1   original_language     8277 non-null   object 
 2   overview              8277 non-null   object 
 3   popularity            8277 non-null   float64
 4   revenue               8277 non-null   float64
 5   runtime               8277 non-null   float64
 6   title                 8277 non-null   object 
 7   vote_average          8277 non-null   float64
 8   genres                8277 non-null   object 
 9   production_companies  8277 non-null   object 
 10  production_countries  8277 non-null   object 
 11  release_year          8277 non-null   float64
 12  return                8277 non-null   float64
dtypes: float64(6), object(7)
memory usage: 905.3+ KB


In [231]:
data = data[['id', 'title', 'genres', 'overview', 'production_companies', 'original_language']]

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7800 entries, 0 to 45374
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    7800 non-null   object 
 1   original_language     7800 non-null   object 
 2   overview              7800 non-null   object 
 3   popularity            7800 non-null   float64
 4   revenue               7800 non-null   float64
 5   runtime               7800 non-null   float64
 6   title                 7800 non-null   object 
 7   vote_average          7800 non-null   float64
 8   genres                7800 non-null   object 
 9   production_companies  7800 non-null   object 
 10  production_countries  7800 non-null   object 
 11  release_year          7800 non-null   float64
 12  return                7800 non-null   float64
dtypes: float64(6), object(7)
memory usage: 853.1+ KB


In [14]:
df = data

In [15]:

# Convertir columnas numéricas a tipos más eficientes
df['popularity'] = df['popularity'].astype('float32')
df['revenue'] = df['revenue'].astype('float32')
df['runtime'] = df['runtime'].astype('float32')
df['vote_average'] = df['vote_average'].astype('float32')
df['release_year'] = df['release_year'].astype('int16')  # Si los años están entre 1900 y 2100
df['return'] = df['return'].astype('float32')

# Convertir columnas de texto a categóricas
df['original_language'] = df['original_language'].astype('category')
df['genres'] = df['genres'].astype('category')
df['production_companies'] = df['production_companies'].astype('category')
df['production_countries'] = df['production_countries'].astype('category')

# Opcional: Eliminar columnas innecesarias si es posible
# df.drop(columns=['id'], inplace=True)  # Ejemplo de eliminación de columna

# Verificar el uso de memoria después de las optimizaciones
print(df.info(memory_usage='deep'))


<class 'pandas.core.frame.DataFrame'>
Index: 7800 entries, 0 to 45374
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   id                    7800 non-null   object  
 1   original_language     7800 non-null   category
 2   overview              7800 non-null   object  
 3   popularity            7800 non-null   float32 
 4   revenue               7800 non-null   float32 
 5   runtime               7800 non-null   float32 
 6   title                 7800 non-null   object  
 7   vote_average          7800 non-null   float32 
 8   genres                7800 non-null   category
 9   production_companies  7800 non-null   category
 10  production_countries  7800 non-null   category
 11  release_year          7800 non-null   int16   
 12  return                7800 non-null   float32 
dtypes: category(4), float32(5), int16(1), object(3)
memory usage: 6.8 MB
None


In [91]:
# Vamos a hacer un tag con las columnas genres, overview, production_companies y original_language
data.head(1)

,id,title,genres,overview,production_companies,original_language
0,862,Toy Story,"['Animation', 'Comedy', 'Family']","Led by Woody, Andy's toys live happily in his ...",['Pixar Animation Studios'],en


In [232]:
data['overview']  = data['overview'].apply(lambda x: x.split())

C:\Users\Jose Quispe}\AppData\Local\Temp\ipykernel_23464\2153982902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['overview']  = data['overview'].apply(lambda x: x.split())


In [42]:
data.head(1)

,id,title,genres,overview,production_companies,original_language
0,862,Toy Story,"['Animation', 'Comedy', 'Family']","[Led, by, Woody,, Andy's, toys, live, happily,...",['Pixar Animation Studios'],en


In [233]:
def collapse(L):
    lista = []
    for i in L:
        lista.append(i.replace(" ",""))
    return lista

In [234]:
import ast

data['genres'] = data['genres'].apply(ast.literal_eval)
data['production_companies'] = data['production_companies'].apply(ast.literal_eval)

C:\Users\Jose Quispe}\AppData\Local\Temp\ipykernel_23464\1142135941.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['genres'] = data['genres'].apply(ast.literal_eval)
C:\Users\Jose Quispe}\AppData\Local\Temp\ipykernel_23464\1142135941.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['production_companies'] = data['production_companies'].apply(ast.literal_eval)


In [235]:
data['genres'] = data['genres'].apply(collapse)
data['production_companies'] = data['production_companies'].apply(collapse)


C:\Users\Jose Quispe}\AppData\Local\Temp\ipykernel_23464\971822252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['genres'] = data['genres'].apply(collapse)
C:\Users\Jose Quispe}\AppData\Local\Temp\ipykernel_23464\971822252.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['production_companies'] = data['production_companies'].apply(collapse)


In [119]:
data.head(1)

,id,original_language,overview,popularity,revenue,runtime,title,vote_average,genres,production_companies,production_countries,release_year,return
0,862,en,"[Led, by, Woody,, Andy's, toys, live, happily,...",21.946943,373554033.0,81.0,Toy Story,7.7,"[Animation, Comedy, Family]",[PixarAnimationStudios],['United States of America'],1995.0,12.451801


In [236]:
#data['tags'] = data['genres'] + data['overview'] + data['production_companies'] 
data['tags'] = data['overview']

C:\Users\Jose Quispe}\AppData\Local\Temp\ipykernel_23464\703194475.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tags'] = data['overview']


In [237]:
data['tags']

0        [Led, by, Woody,, Andy's, toys, live, happily,...
1        [When, siblings, Judy, and, Peter, discover, a...
2        [A, family, wedding, reignites, the, ancient, ...
4        [Just, when, George, Banks, has, recovered, fr...
5        [Obsessive, master, thief,, Neil, McCauley, le...
                               ...                        
45331    [In, this, family-friendly, action, reboot, of...
45337    [In, her, filthy, cell, in, Newgate, prison, M...
45341    [The, Sublet, is, a, suspense, driven, psychol...
45363    [It's, Halloween, in, the, 100, Acre, Wood,, a...
45374    [Yet, another, version, of, the, classic, epic...
Name: tags, Length: 8277, dtype: object

In [238]:
data = data[['id', 'title', 'tags']]

In [96]:
data.head(1)

,id,title,tags
0,862,Toy Story,"[Led, by, Woody,, Andy's, toys, live, happily,..."


In [239]:
data['tags'] = data['tags'].apply(lambda x:" ".join(x))

In [240]:
data.head(2)

,id,title,tags
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...


In [ ]:
#data.to_csv("peliculas_para_el_modelo.csv", index=False)

In [241]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')

In [242]:
vector = cv.fit_transform(data['tags']).toarray()

In [63]:
vector.shape

(15431, 5000)

In [243]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vector)

In [102]:
similarity

array([[1.        , 0.02597622, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02597622, 1.        , 0.06131393, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.06131393, 1.        , ..., 0.        , 0.06579517,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.05248639,
        0.        ],
       [0.        , 0.        , 0.06579517, ..., 0.05248639, 1.        ,
        0.07597372],
       [0.        , 0.        , 0.        , ..., 0.        , 0.07597372,
        1.        ]])

In [244]:
def recomendar(pelicula):
    index = data[data['title'] == pelicula].index[0]

    distancias = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x:x[1])

    for i in distancias[1:6]:
        print(data.iloc[i[0]].title)

In [245]:
recomendar('Jumanji')

Table No. 21
Beta Test
Brainscan
Wreck-It Ralph
Nirvana


In [203]:
df = data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
import pandas as pd

# Supongamos que ya tienes tu DataFrame `df`

# Paso 1: Asegúrate de que `overview` esté en formato de texto
df['overview'] = df['overview'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x).fillna('')

# Paso 2: Convierte las listas en texto para `genres`, `production_companies` y `production_countries`
df['genres'] = df['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df['production_companies'] = df['production_companies'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df['production_countries'] = df['production_countries'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Paso 3: Verifica si las celdas están en formato de texto
df['genres'] = df['genres'].astype(str)
df['production_companies'] = df['production_companies'].astype(str)
df['production_countries'] = df['production_countries'].astype(str)



# Vectorización de `overview` usando TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['overview'])

# Vectorización de `genres` usando MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genres_matrix = mlb.fit_transform(df['genres'].str.split(', '))

# Normalización de `vote_average` y `release_year`
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[['vote_average', 'release_year']])

# Concatenar todas las características en una matriz final
from scipy.sparse import hstack
feature_matrix = hstack([tfidf_matrix, genres_matrix, scaled_features])

# Ahora puedes calcular la similitud del coseno con la matriz final
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(feature_matrix)


In [16]:
df.to_csv("peliculas_para_el_modelo2.csv")

In [206]:
def get_recommendations(title, df, cosine_sim, top_n=5):
    # Obtener el índice de la película que coincide con el título
    try:
        idx = df[df['title'] == title].index[0]
    except IndexError:
        return f"No se encontró la película '{title}' en la base de datos."

    # Obtener las puntuaciones de similitud de todas las películas con la película dada
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar las películas en base a la similitud (de mayor a menor) y seleccionar las top_n
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los índices de las películas más similares (excluyendo la misma película)
    sim_indices = [i[0] for i in sim_scores[1:top_n+1]]

    # Retornar los títulos de las películas recomendadas
    recommended_titles = df.iloc[sim_indices]['title'].tolist()
    return recommended_titles


In [247]:
title = "Jumanji"
recommendations = get_recommendations(title, df, cosine_sim)
print("Películas recomendadas:", recommendations)

Películas recomendadas: ['Peter Pan', 'Neverland', "Harry Potter and the Philosopher's Stone", 'Jack the Giant Killer', 'Hook']


In [217]:
data['title']

0                                            Toy Story
1                                              Jumanji
2                                     Grumpier Old Men
4                          Father of the Bride Part II
5                                                 Heat
                             ...                      
45331                      Cop and a Half: New Recruit
45337    The Fortunes and Misfortunes of Moll Flanders
45341                                       The Sublet
45363                 Pooh's Heffalump Halloween Movie
45374                                       Robin Hood
Name: title, Length: 8277, dtype: object

In [1]:
df.info()

NameError: name 'df' is not defined